In [ ]:
# Import libraries
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.orm import Session
from ast import literal_eval

In [ ]:
# Read in the CSVs using Pandas
#National Parks
national_parks_df = pd.read_csv("../Resources/national_parks.csv")

#Trails
csv_file = "../Resources/National Park Trail Data.csv"
trail_df = pd.read_csv(csv_file)

#Species
species = "../Resources/National_Parks_Species.csv"
species_df = pd.read_csv(species, keep_default_na=False)

#Visitation
visitors_file = "../Resources/All National Parks Visitation 1904-2016.csv"
visitors_df = pd.read_csv(visitors_file)

#Bear Attacks
bear_attacks_df = pd.read_csv("../Resources/north_america_bear_killings.csv")

# National Parks Transformation

In [ ]:
# Read in the national_parks.csv into a dataframe
national_parks_df.head()

In [ ]:
# Look at the column names
national_parks_df.columns

In [ ]:
# Look at the column data types
national_parks_df.dtypes

In [ ]:
# Need to rename the columns to match the table columns in nation_parks_db
national_parks_df = national_parks_df.rename(columns={"Park Code":"park_code","Park Name":"park_name",
                                                      "State":"state","Acres":"acres","Latitude":"latitude",
                                                      "Longitude":"longitude"})
national_parks_df.columns

In [ ]:
# Verify no duplicates
national_parks_df.drop_duplicates().shape

In [ ]:
national_parks_df.index.name = "id"
national_parks_df.head()

# Trails CSV Transformation

In [ ]:
trail_df['_geoloc'] = trail_df['_geoloc'].astype('str').apply(literal_eval)
display(trail_df[trail_df['area_name'] == 'Clayton Co International Park, Jonesboro GA'])
display(trail_df[trail_df['area_name'] == 'Fort Hunt National Park'])
display(trail_df[trail_df['area_name'] == 'Wolf Trap National Park for the Performing Arts'])
trail_df.drop([1219,2475,2618,2629,2524],inplace=True)
trail_df.reset_index(inplace=True,drop=True)

replace_dict = {'Great Sand Dunes National Park and Preserve':'Great Sand Dunes National Park',
 'Congaree National Park Wilderness':'Congaree National Park',
 'Hawaii Volcanoes National Park':"Hawai'i Volcanoes National Park",
 'Haleakala National Park':"Haleakalā National Park"}

trail_df.replace(replace_dict,inplace=True)

In [ ]:
trail_df.head()

In [ ]:
features_df=trail_df[['trail_id','area_name','features']]

In [ ]:
features_df['features'] = features_df['features'].astype('str').apply(literal_eval)
features_df = features_df.explode('features')

In [ ]:
features_df.dropna(inplace=True)
features_df.reset_index(inplace=True, drop=True)
features_df.rename(columns={'area_name':'park_name'},inplace=True)

In [ ]:
features_df.head()

In [ ]:
activities_df=trail_df[['trail_id','area_name','activities']]

In [ ]:
activities_df['activities'] = activities_df['activities'].astype('str').apply(literal_eval)
activities_df = activities_df.explode('activities')
activities_df.dropna(inplace=True)
activities_df.reset_index(inplace=True, drop=True)
activities_df.rename(columns={'area_name':'park_name'},inplace=True)
activities_df.head()

In [ ]:
trail_df_clean = trail_df.drop(columns=['features','activities'])

In [ ]:
trail_df_clean.head()

In [ ]:
trail_df_clean['latitude'] = ''
trail_df_clean['longitude'] = ''
for i,row in trail_df_clean.iterrows():
    trail_df_clean.iloc[i,16] = trail_df_clean.iloc[i,6]['lat']
    trail_df_clean.iloc[i,17] = trail_df_clean.iloc[i,6]['lng']
    

In [ ]:
trail_df_clean.drop(columns='_geoloc',inplace=True)

In [ ]:
trail_df_clean.dropna(inplace=True,)

In [ ]:
trail_df_clean.reset_index(inplace=True, drop=True)

In [ ]:
trail_df_clean.rename(columns={'name':'trail_name','city_name':'city','area_name':'park_name','state_name':'state','country_name':'country'},inplace=True)

In [ ]:
trail_df_clean.head()

In [ ]:
activities_df['id'] = activities_df.index + 1
features_df['id'] = features_df.index +1

# Species CSV Transformation

In [ ]:
# Printing csv file
species_df.head()

In [ ]:
# Dropping unwanted columns
# Printing the new DataFrame
species_df = species_df.drop(columns=['Order', 'Family', 'Record Status', 'Nativeness', 'Seasonality', 'Conservation Status', 'Unnamed: 13'])
species_df.head()

In [ ]:
# Keeping species that are only "Present" - Keeping rows where value in "Occurrence" column is equal to "Present"
# Printing new DataFrame
species_df = species_df.loc[species_df['Occurrence'] == 'Present']
species_df.head()

In [ ]:
# Renaming columns
species_df = species_df.rename(columns={'Species ID': 'species_id', 'Park Name': 'park_name', 'Category': 'category', 'Scientific Name': 'scientific_name', 'Occurrence': 'occurrence', 'Common Names':'common_names', 'Occurence':'occurence', 'Abundance':'abundance'})
species_df.head()

In [ ]:
# Checking for duplicates
# No duplicates
species_df.duplicated()

In [ ]:
# Resetting index
# Printing new DataFrame
species_df = species_df.reset_index(drop=True)
species_df.head()

In [ ]:
# Renaming the index to "id"
# Printing the new DataFrame
species_df.index.name = 'id'
species_df.head()

# Parks Visitation CSV Transformation

In [ ]:
# Load DF
visitors_df.head()

In [ ]:
# Grab only National Parks from the DataFrame
visitors_df = visitors_df.loc[visitors_df["Unit Type"] == "National Park"]

# filter out rows that do not contain a numeric year
visitors_df = visitors_df.loc[visitors_df["YearRaw"] != "Total"]

# Create filtered dataframe with specific columns
visitors_cols = ["Region", "State", "Unit Name", "Visitors", "YearRaw"]
visitors_transformed= visitors_df[visitors_cols].copy()



# Rename the column headers for clarification
visitors_transformed = visitors_transformed.rename(columns={"Region":"region",
                                                            "State":"state",
                                                            "Unit Name": "park_name",
                                                            "Visitors":"visitors",
                                                            "YearRaw": "year"})
# Reset the index
visitors_transformed.reset_index(drop=True, inplace=True)

# Create the ID column based off of the index
visitors_transformed['id'] = visitors_transformed.index + 1

# Set the index to the ID
visitors_transformed.set_index('id',drop=True, inplace=True)

visitors_transformed.head()

# Bear Attacks CSV Transformation

In [ ]:
#inspect the columns
bear_attacks_df.columns

In [ ]:
#age column has a leading space - remove that and make all others lower case
bear_attacks_df = bear_attacks_df.rename(columns={'Name': 'name', ' age': 'age', 'Date': 'date', 'Type': 'type', 'Location': 'location', 'Description': 'description', 'Type of bear': 'bear_type'})


In [ ]:
#we only care about attacks in National Parks - filter for those - these will include Canada's National Parks as well as USA National Parks
in_parks = bear_attacks_df['location'].str.contains('National Park', case=True, regex=False)

#select the relevant columns
in_parks_df = bear_attacks_df[in_parks]
in_parks_df = in_parks_df[['name', 'age', 'date', 'type', 'location', 'description', 'bear_type']]


In [ ]:
#check for any duplicate records
in_parks_df.shape

In [ ]:
#there are no duplicate records
in_parks_df.drop_duplicates().shape


In [ ]:
#reset the index - this will be the sql primary key
in_parks_df = in_parks_df.reset_index(drop=True)
#give index column name 'id'
in_parks_df.index.name = 'id'

In [ ]:
in_parks_df

In [ ]:
in_parks_df['location'] = in_parks_df['location'].str.split(',').str[0]
in_parks_df

# Data Loads to Postgres

In [ ]:
# Open database connection
db_conn = "postgres:postgres@localhost:5432/national_parks_db"
engine = create_engine(f"postgresql://{db_conn}")

# Check the table names for accuracy
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Load the National Parks dataframe into the postgres table
national_parks_df.to_sql(name="national_parks", con=engine, if_exists='append', index=True)

# Verify the data was loaded in the table
pd.read_sql_query("select * from national_parks", con=engine).head()


In [ ]:
# Load the Trails database tables
trail_df_clean.to_sql(name='park_trails', con=engine, if_exists='append', index=True)
activities_df.to_sql(name='trail_activities', con=engine, if_exists='append', index=True)
features_df.to_sql(name='trail_features', con=engine, if_exists='append', index=True)

# Verify the data was loaded in the table
pd.read_sql_query("select * from park_trails", con=engine).head()

In [ ]:
# Loading Species DataFrame into database
species_df.to_sql(name='species', con=engine, if_exists='append', index=True)

# Verify the data was loaded in the table
pd.read_sql_query("select * from species", con=engine).head()

In [ ]:
# Load Visitors dataframe into the database
visitors_transformed.to_sql(name='parks_visitation', con=engine, if_exists='append', index=True)

# Verify the data was loaded in the table
pd.read_sql_query("select * from parks_visitation", con=engine).head()

In [ ]:
#write Bear Attack data to sql
in_parks_df.to_sql(name='bear_attacks', con=engine, if_exists='append', index=True)

# Verify the data was loaded in the table
pd.read_sql_query("select * from bear_attacks", con=engine).head()